In [1]:
import torch
import torch.nn as nn

class DP_GSD(nn.Module):

    def __init__(self):
        super(DP_GSD, self).__init__()
        
        self.flat = nn.Flatten()
        self.l1 = nn.Linear(784, 392)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(392, 10)
        self.sm = nn.Softmax()

    def forward(self, x):
        
        x = self.flat(x)
        x = self.l1(x)
        x = self.relu(x)
        x = self.l2(x)
        x = self.sm(x)
        
        return x

In [2]:
MAX_GRAD_NORM = 1.2
EPSILON = 50.0
DELTA = 1e-5
EPOCHS = 10

LR = 1e-3

BATCH_SIZE = 512
MAX_PHYSICAL_BATCH_SIZE = 128

In [3]:
from torchvision.datasets import MNIST
import torchvision.transforms as transforms



DATA_ROOT = '../mnist'

transform = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = MNIST(
    root=DATA_ROOT, train=True, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
)

test_dataset = MNIST(
    root=DATA_ROOT, train=False, download=True, transform=transform)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
)

print(len(train_dataset))
dataiter = iter(train_loader)
images, labels = next(dataiter)
images[0].size()

60000


torch.Size([1, 28, 28])

In [4]:
from torchvision import models

model = models.resnet50(num_classes=10)

from opacus.validators import ModuleValidator

model = ModuleValidator.fix(model)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device)

ModuleValidator.validate(model, strict=False)

[]

In [5]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters(), lr=LR)

def accuracy(preds, labels):
    return (preds == labels).mean()

from opacus import PrivacyEngine

privacy_engine = PrivacyEngine()

#model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
#    module=model,
#    optimizer=optimizer,
#    data_loader=train_loader,
#    epochs=EPOCHS,
#    target_epsilon=EPSILON,
#    target_delta=DELTA,
#    max_grad_norm=MAX_GRAD_NORM,
#)

#print(f"Using sigma={optimizer.noise_multiplier} and C={MAX_GRAD_NORM}")

/home/knil/Nextcloud2/RAstuff/opacus/opacus/privacy_engine.py:96: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(


In [6]:
import numpy as np
import torch.optim as optim
from opacus.utils.batch_memory_manager import BatchMemoryManager


def train(model, train_loader, optimizer, epoch, device):
    model.train()
    criterion = nn.CrossEntropyLoss()

    losses = []
    top1_acc = []
    
    with BatchMemoryManager(
        data_loader=train_loader, 
        max_physical_batch_size=MAX_PHYSICAL_BATCH_SIZE, 
        optimizer=optimizer
    ) as memory_safe_data_loader:

        for i, (images, target) in enumerate(memory_safe_data_loader):   
            optimizer.zero_grad()
            images = images.to(device)
            target = target.to(device)

            # compute output
            output = model(images)
            loss = criterion(output, target)

            preds = np.argmax(output.detach().cpu().numpy(), axis=1)
            labels = target.detach().cpu().numpy()

            # measure accuracy and record loss
            acc = accuracy(preds, labels)

            losses.append(loss.item())
            top1_acc.append(acc)

            loss.backward()
            optimizer.step()
            

            if (i+1) % 200 == 0:
                epsilon = privacy_engine.get_epsilon(DELTA)
                print(
                    f"\tTrain Epoch: {epoch} \t"
                    f"Loss: {np.mean(losses):.6f} "
                    f"Acc@1: {np.mean(top1_acc) * 100:.6f} "
                    f"(ε = {epsilon:.2f}, δ = {DELTA})"
                )
    return acc, epsilon
    

In [7]:
def test(model, test_loader, device):
    model.eval()
    criterion = nn.CrossEntropyLoss()
    losses = []
    top1_acc = []

    with torch.no_grad():
        for images, target in test_loader:
            images = images.to(device)
            target = target.to(device)

            output = model(images)
            loss = criterion(output, target)
            preds = np.argmax(output.detach().cpu().numpy(), axis=1)
            labels = target.detach().cpu().numpy()
            acc = accuracy(preds, labels)

            losses.append(loss.item())
            top1_acc.append(acc)

    top1_avg = np.mean(top1_acc)

    print(
        f"\tTest set:"
        f"Loss: {np.mean(losses):.6f} "
        f"Acc: {top1_avg * 100:.6f} "
    )
    return np.mean(top1_acc)


from tqdm.notebook import tqdm

report = []

#for epoch in tqdm(range(EPOCHS), desc="Epoch", unit="epoch"):
#    results = train(model, train_loader, optimizer, epoch + 1, device)
#    report.append(results)

In [8]:
acc = []
e = []
for i in report:
    acc.append(i[0])
    e.append(i[1])

In [9]:
args ={
            "a1":0.1,
            "a3":0.1,
            "a4":0.1,
            "lam":5,
            "moment":1,
            "theta":0.05,
            'k':1,
            'mu':0,
            'sigma':0.1,
            'a':0,
            'b':1,
            'u':1,
            'l':0.1,
            'epsilon':1,
            'max_grad_norm': 1,
        }

In [ ]:
from opacus import PrivacyEngine
runs = []
trun = []

model = DP_GSD()
from opacus.validators import ModuleValidator

model = ModuleValidator.fix(model)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device)
    
privacy_engine = PrivacyEngine(accountant = 'rdp_plrv')
    
optimizer = optim.RMSprop(model.parameters(), lr=LR)

model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
        module=model,
        optimizer=optimizer,
        data_loader=train_loader,
        noise_multiplier = 1,
        epochs=EPOCHS,
        target_epsilon=0.5,
        target_delta=DELTA,
        max_grad_norm=1,
        PLRV_args=args,
)
    
plrv_report_acc = []
plrv_report_ep = []

for epoch in tqdm(range(EPOCHS), desc="Epoch", unit="epoch"):
    acc, ep = train(model, train_loader, optimizer, epoch + 1, device)
    plrv_report_acc.append(acc)
    plrv_report_ep.append(ep)
    
del model
del optimizer
del results
torch.cuda.empty_cache()

In [10]:
from opacus import PrivacyEngine
runs = []
trun = []

model = DP_GSD()
from opacus.validators import ModuleValidator

model = ModuleValidator.fix(model)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device)
    
privacy_engine = PrivacyEngine(accountant = 'rdp')
    
optimizer = optim.RMSprop(model.parameters(), lr=LR)

model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
        module=model,
        optimizer=optimizer,
        data_loader=train_loader,
        #noise_multiplier = 1,
        epochs=EPOCHS,
        target_epsilon=0.5,
        target_delta=DELTA,
        max_grad_norm=1,
        #PLRV_args=args,
)
    
rdp_report_acc = []
rdp_report_ep = []

for epoch in tqdm(range(EPOCHS), desc="Epoch", unit="epoch"):
    acc, ep = train(model, train_loader, optimizer, epoch + 1, device)
    rdp_report_acc.append(acc)
    rdp_report_ep.append(ep)
    

del model
del optimizer
del results
torch.cuda.empty_cache()

/home/knil/Nextcloud2/RAstuff/opacus/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?epoch/s]

/home/knil/anaconda3/lib/python3.9/site-packages/torch/nn/modules/module.py:1344: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/tmp/ipykernel_365362/3959651094.py:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.sm(x)


	Train Epoch: 1 	Loss: 2.247524 Acc@1: 53.328867 (ε = 0.11, δ = 1e-05)
	Train Epoch: 1 	Loss: 2.170614 Acc@1: 61.743939 (ε = 0.12, δ = 1e-05)
	Train Epoch: 2 	Loss: 1.913602 Acc@1: 72.595604 (ε = 0.13, δ = 1e-05)


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt   
plt.plot(rdp_report_ep, rdp_report_acc, 'ro-', label="Gaussian")
plt.plot(plrv_report_ep, plrv_report_acc, 'bo-', label="PLRV")
plt.legend(loc="lower right")
plt.xlabel("Epsilon")
plt.ylabel("Accuracy")
plt.title("Training Accuracy vs epsilon (Gaussian at 0.5)")
plt.show()

In [ ]:
import matplotlib.pyplot as plt   
plt.plot(range(10), rdp_report_acc, 'ro-', label="Gaussian")
plt.plot(range(10), plrv_report_acc, 'bo-', label="PLRV")
plt.legend(loc="lower right")
plt.xlabel("Epsilon")
plt.ylabel("Accuracy")
plt.title("Training Accuracy vs epsilon (Gaussian at 0.5)")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
labels = [0.2, 0.3, 0.5, 1, 2]
for j in range(len(runs)):
    acc = []
    e = []
    for i in runs[j]:
        acc.append(i[0])
        e.append(i[1])
    
    plt.plot(range(10), e, label=labels[j])
    
plt.legend(loc="lower right")
plt.xticks(range(10))
plt.xlabel("Epoch")
plt.ylabel("Epsilon")
plt.title("Training Accuracy per Epoch for different values of Maximum Epsilon")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
labels = ["run 1","run 2","run 3","run 4","run 5"]
for j in range(len(runs)):
    acc = []
    e = []
    for i in runs[j]:
        acc.append(i[0])
        e.append(i[1])
    
    plt.plot(acc, e, label=labels[j])
    
plt.legend(loc="lower right")
#plt.xticks(range(10))
plt.xlabel("Accuracy")
plt.ylabel("Epsilon")
plt.title("Training Accuracy per Epoch for different values of Maximum Epsilon")
plt.show()

In [ ]:
x = range
y=[]
for i in runs:
    y.append(i[-1][0])

plt.xlabel("Epsilon")
plt.ylabel("Accuracy")
plt.title("Final Accuracy vs Epsilon")
plt.plot(x, trun, 'ro-', label="Testing Acc")
plt.legend(loc="lower right")
plt.show()

In [ ]:
runs = []
trun = []

for i in [0.2, 0.3, 0.5, 1, 2]:
    model = DP_GSD()

    from .opacus.validators import ModuleValidator

    model = ModuleValidator.fix(model)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = model.to(device)
    
    privacy_engine = PrivacyEngine()
    
    optimizer = optim.RMSprop(model.parameters(), lr=LR)

    model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
        module=model,
        optimizer=optimizer,
        data_loader=train_loader,
        epochs=i,
        target_epsilon=1,
        target_delta=DELTA,
        max_grad_norm=2,
    )
    
    report = []

    for epoch in tqdm(range(i), desc="Epoch", unit="epoch"):
        results = train(model, train_loader, optimizer, epoch + 1, device)
        report.append(results)
    
    runs.append(report)
    trun.append(test(model, test_loader, device))
    del model

In [ ]:
x = [0.2, 0.3, 0.5, 1, 2]
y=[]
for i in runs:
    y.append(i[-1][0])

plt.xlabel("Epsilon")
plt.ylabel("Accuracy")
#plt.title("Final Accuracy vs Epochs with Epsilon = 2")
plt.plot(x, y, 'bo-', label="Training Acc")
plt.plot(x, trun, 'ro-', label="Testing Acc")
plt.legend(loc="lower right")
plt.show()

In [ ]:
import torch

In [ ]:
from opacus.optimizers.optimizer import _check_processed_flag, _mark_as_processed, DPOptimizer
from torch.distributions.laplace import Laplace
from torch.distributions.gamma import Gamma
from torch.distributions.normal import Normal
from torch.distributions.uniform import Uniform
from torch import stack, zeros, einsum
from opacus.optimizers.utils import params
from torch import nn
from torch.optim import Optimizer
from scipy.stats import truncnorm, expon

class PLRVDPOptimizer():
    """
    Implementation of PLRV first noise mechanism.
    """
        
    def make_noise(self, args):
        self.args = args
        self.k = self.args['k']
        self.theta = self.args['theta']
        self.mu = self.args['mu']
        self.sigma = self.args['sigma']
        self.a = self.args['a']
        self.b = self.args['b']
        self.l = self.args['l']
        self.u = self.args['u']
        self.clip = self.args['max_grad_norm']
        self.lam = self.args['lam']
        
        a_transformed, b_transformed = (self.l - self.mu) / self.sigma, (self.u - self.mu) / self.sigma
        
        self.gamma = Gamma(
          concentration = self.k, rate = self.theta
          )
        self.normal= truncnorm(
          a_transformed, b_transformed, loc=self.mu, scale=self.sigma
          )
        self.uniform = Uniform(
          low = self.a, high = self.b
          )
        self.expon = expon(loc=0, scale = 1/self.lam)
    
    
    def add_noise(self):
          
        for p in self.params:
            _check_processed_flag(p.summed_grad)
            
            laplace = self.get_laplace()
            noise = laplace.sample(p.summed_grad.shape).to(p.summed_grad.device)
            p.grad = p.summed_grad + noise

            _mark_as_processed(p.summed_grad)
            
    def get_linear_combination(self):
        gam = self.gamma.sample()
        uni = self.uniform.sample()
        t_norm = self.normal.rvs(size=1)[0]  
        exp = self.expon.rvs(size=1)[0]
        return 1/(self.args['a1']*gam+self.args['a3']*exp+self.args['a4']*uni)
        
    def get_laplace(self):
        return Laplace(loc=0, scale=self.get_linear_combination())
        
    def clip_and_accumulate(self):
        """
        Performs gradient clipping.
        Stores clipped and aggregated gradients into `p.summed_grad```
        """

        if len(self.grad_samples[0]) == 0:
            # Empty batch
            per_sample_clip_factor = zeros(
                (0,), device=self.grad_samples[0].device
            )
        else:
            per_param_norms = [
                g.reshape(len(g), -1).norm(1, dim=-1) for g in self.grad_samples
            ]
            per_sample_norms = stack(per_param_norms, dim=1).norm(1, dim=1)
            per_sample_clip_factor = (
                self.max_grad_norm / (per_sample_norms + 1e-6)
            ).clamp(max=1.0)

        for p in self.params:
            _check_processed_flag(p.grad_sample)
            grad_sample = self._get_flat_grad_sample(p)
            grad = einsum("i,i...", per_sample_clip_factor, grad_sample)

            if p.summed_grad is not None:
                p.summed_grad += grad
            else:
                p.summed_grad = grad

            _mark_as_processed(p.grad_sample)

In [ ]:
noise = PLRVDPOptimizer()
noise.make_noise(args)
laplace = noise.get_laplace()
laplace.sample((1,1000)).tolist()[0]

In [ ]:
import matplotlib.pyplot as plt
samples = []
for i in range(1000):
    laplace = noise.get_laplace()
    samples.append(laplace.sample().tolist())
    
print(samples)

In [ ]:
plt.plot(range(1000), samples)
plt.show()

In [ ]:
from opacus.optimizers.optimizer import _check_processed_flag, _mark_as_processed, DPOptimizer
from torch.distributions.laplace import Laplace
from torch.distributions.gamma import Gamma
from torch.distributions.normal import Normal
from torch.distributions.uniform import Uniform
from torch import stack, zeros, einsum
from opacus.optimizers.utils import params
from torch import nn
from torch.optim import Optimizer
from scipy.stats import truncnorm, expon

class PLRVDPOptimizer():
    """
    Implementation of PLRV first noise mechanism.
    """
        
    def make_noise(self, args):
        self.args = args
        self.k = self.args['k']
        self.theta = self.args['theta']
        self.mu = self.args['mu']
        self.sigma = self.args['sigma']
        self.a = self.args['a']
        self.b = self.args['b']
        self.l = self.args['l']
        self.u = self.args['u']
        self.clip = self.args['max_grad_norm']
        self.lam = self.args['lam']
        
        a_transformed, b_transformed = (self.l - self.mu) / self.sigma, (self.u - self.mu) / self.sigma
        
        self.gamma = Gamma(
          concentration = self.k, rate = self.theta
          )
        self.normal= truncnorm(
          a_transformed, b_transformed, loc=self.mu, scale=self.sigma
          )
        self.uniform = Uniform(
          low = self.a, high = self.b
          )
        self.expon = expon(loc=0, scale = 1/self.lam)
    
    
    def add_noise(self):
          
        for p in self.params:
            _check_processed_flag(p.summed_grad)
            
            laplace = self.get_laplace()
            noise = laplace.sample(p.summed_grad.shape).to(p.summed_grad.device)
            p.grad = p.summed_grad + noise

            _mark_as_processed(p.summed_grad)
            
    def get_linear_combination(self):
        gam = self.gamma.sample()
        uni = self.uniform.sample()
        t_norm = self.normal.rvs(size=1)[0]  
        exp = self.expon.rvs(size=1)[0]
        return 1/(self.args['a1']*gam+self.args['a3']*exp+self.args['a4']*uni)
        
    def get_laplace(self):
        return Laplace(loc=0, scale=self.get_linear_combination())
        
    def clip_and_accumulate(self):
        """
        Performs gradient clipping.
        Stores clipped and aggregated gradients into `p.summed_grad```
        """

        if len(self.grad_samples[0]) == 0:
            # Empty batch
            per_sample_clip_factor = zeros(
                (0,), device=self.grad_samples[0].device
            )
        else:
            per_param_norms = [
                g.reshape(len(g), -1).norm(1, dim=-1) for g in self.grad_samples
            ]
            per_sample_norms = stack(per_param_norms, dim=1).norm(1, dim=1)
            per_sample_clip_factor = (
                self.max_grad_norm / (per_sample_norms + 1e-6)
            ).clamp(max=1.0)

        for p in self.params:
            _check_processed_flag(p.grad_sample)
            grad_sample = self._get_flat_grad_sample(p)
            grad = einsum("i,i...", per_sample_clip_factor, grad_sample)

            if p.summed_grad is not None:
                p.summed_grad += grad
            else:
                p.summed_grad = grad

            _mark_as_processed(p.grad_sample)

In [ ]:
args ={
            "a1":0.1,
            "a3":0.1,
            "a4":0.1,
            "lam":5,
            "moment":1,
            "theta":0.5,
            'k':1,
            'mu':0,
            'sigma':0.5,
            'a':1,
            'b':2,
            'u':1,
            'l':0.1,
            'epsilon':1,
            'max_grad_norm': 1,
        }

In [ ]:
noise = PLRVDPOptimizer()
noise.make_noise(args)
laplace = noise.get_laplace()
laplace.sample((1,1000)).tolist()[0]

import matplotlib.pyplot as plt
samples = []
for i in range(1000):
    laplace = noise.get_laplace()
    samples.append(laplace.sample().tolist())
    
print(samples)

In [ ]:
plt.plot(range(1000), samples)
plt.show()